In [44]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
import re
from textblob import TextBlob
from afinn import Afinn
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# Preprocess data

* Merge df_reviews and df_meta on asin (product unique number)
* Clean textual data

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')
# reviews_original = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/df_reviews.csv')
reviews_original = pd.read_csv('df_reviews.csv')
reviews_original.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Rating,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,1,3.0,5,True,2013-10-23,A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400
1,5,4.0,NaN,True,2013-06-05,A3IYSOTP3HA77N,0380709473,{'Format:': ' Kindle Edition'},B. Marks,I read this probably 50 years ago in my youth ...,Above average mystery,1370390400
2,6,5.0,NaN,True,2016-06-27,A11SXV34PZUQ5E,0380709473,{'Format:': ' Kindle Edition'},Tom C.,I read every Perry mason book voraciously. Fin...,Lam is cool!,1466985600
3,7,5.0,NaN,True,2015-07-30,A2AUQM1HT2D5T8,0380709473,{'Format:': ' Kindle Edition'},ema,I love this series of Bertha and Lamb.. Great...,Five Stars,1438214400
4,8,5.0,NaN,True,2015-02-16,A3UD8JRWLX6SRX,0380709473,{'Format:': ' Paperback'},Michael O.,Great read!,Five Stars,1424044800


In [7]:
reviews = reviews_original.copy() # make a copy so that we don't need to load the data again

In [8]:
# meta_original = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/df_meta.csv')
meta_original = pd.read_csv('df_meta.csv')

In [9]:
meta = meta_original.copy()

In [10]:
df_merge = pd.merge(meta, reviews, on='asin', how='inner')

In [11]:
df_merge.head()

,Unnamed: 0_x,index,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details,rankElectronics,Unnamed: 0_y,Rating,vote,verified,reviewTime,reviewerID,style,reviewerName,reviewText,summary,unixReviewTime
0,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotap...","['>#15,985 in Electronics (See Top 100 in Elec...",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5654,5.0,NaN,True,2014-09-08,A2PUOBJDETBPQB,NaN,Nicholas in Seattle,Tapes are exactly what I wanted. I'll be using...,Maxell Tapes,1410134400
1,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotap...","['>#15,985 in Electronics (See Top 100 in Elec...",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5655,5.0,NaN,True,2015-07-13,A2Z9HS4YPRDUWR,NaN,Mr. William H. Hamilton,LOVE THIS PRODUCT...NOT TOO READILY AVAILABLE ...,STILL WORKS GREAT!,1436745600
2,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotap...","['>#15,985 in Electronics (See Top 100 in Elec...",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5656,5.0,NaN,True,2015-06-24,AAGXCOQ8HSR95,NaN,ronald h sarnacki,very good qualiry,Five Stars,1435104000
3,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotap...","['>#15,985 in Electronics (See Top 100 in Elec...",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5657,1.0,NaN,True,2015-01-09,A28YYCYMAX19BX,NaN,chiefknisley--retired,no good,One Star,1420761600
4,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotap...","['>#15,985 in Electronics (See Top 100 in Elec...",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5658,5.0,NaN,True,2014-11-22,A2HLEPPTHZDW1M,NaN,john mcdonald,The old camcorder worked great! I just needed...,The old camcorder still does what I need.,1416614400


In [12]:
df_merge.shape

(315686, 31)

## Data cleaning
### Replace non-letter characters

In [13]:
## clean data

# remove number and special characters
df_merge['reviewClean'] = df_merge['reviewText'].replace("[^a-zA-Z]+", " ", regex = True)
df_merge['summaryClean'] = df_merge['summary'].replace("[^a-zA-Z]+", " ", regex = True)

# lower case
df_merge['reviewClean'] = df_merge['reviewClean'].str.lower()
df_merge['summaryClean'] = df_merge['summaryClean'].str.lower()

# remove white spaces
df_merge['reviewClean'] = df_merge['reviewClean'].str.strip()
df_merge['summaryClean'] = df_merge['summaryClean'].str.strip()

# fill empty review with na
df_merge['reviewClean'].fillna('', inplace= True)
df_merge['summaryClean'].fillna('', inplace= True)

In [14]:
df_merge.head()

,Unnamed: 0_x,index,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details,rankElectronics,Unnamed: 0_y,Rating,vote,verified,reviewTime,reviewerID,style,reviewerName,reviewText,summary,unixReviewTime,reviewClean,summaryClean
0,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotap...","['>#15,985 in Electronics (See Top 100 in Elec...",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5654,5.0,NaN,True,2014-09-08,A2PUOBJDETBPQB,NaN,Nicholas in Seattle,Tapes are exactly what I wanted. I'll be using...,Maxell Tapes,1410134400,tapes are exactly what i wanted i ll be using ...,maxell tapes
1,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotap...","['>#15,985 in Electronics (See Top 100 in Elec...",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5655,5.0,NaN,True,2015-07-13,A2Z9HS4YPRDUWR,NaN,Mr. William H. Hamilton,LOVE THIS PRODUCT...NOT TOO READILY AVAILABLE ...,STILL WORKS GREAT!,1436745600,love this product not too readily available an...,still works great
2,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotap...","['>#15,985 in Electronics (See Top 100 in Elec...",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5656,5.0,NaN,True,2015-06-24,AAGXCOQ8HSR95,NaN,ronald h sarnacki,very good qualiry,Five Stars,1435104000,very good qualiry,five stars
3,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotap...","['>#15,985 in Electronics (See Top 100 in Elec...",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5657,1.0,NaN,True,2015-01-09,A28YYCYMAX19BX,NaN,chiefknisley--retired,no good,One Star,1420761600,no good,one star
4,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotap...","['>#15,985 in Electronics (See Top 100 in Elec...",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5658,5.0,NaN,True,2014-11-22,A2HLEPPTHZDW1M,NaN,john mcdonald,The old camcorder worked great! I just needed...,The old camcorder still does what I need.,1416614400,the old camcorder worked great i just needed s...,the old camcorder still does what i need


### Remove stop words

In [15]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [16]:
excluded = ['no', 'not']

In [17]:
# remove stopwords
# df_merge['reviewClean'] = df_merge['reviewClean'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df_merge['reviewClean'] = df_merge['reviewClean'].apply(lambda x: " ".join(x for x in x.split() if (x not in stop) or (x in excluded)))
df_merge['summaryClean'] = df_merge['summaryClean'].apply(lambda x: " ".join(x for x in x.split() if (x not in stop) or (x in excluded)))

In [18]:
pd.set_option("max_colwidth", 1000)

In [ ]:
df_merge.head()

,Unnamed: 0_x,index,category,tech1,description,fit,title,also_buy,tech2,brand,...,verified,reviewTime,reviewerID,style,reviewerName,reviewText,summary,unixReviewTime,reviewClean,summaryClean
0,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,...,True,2014-09-08,A2PUOBJDETBPQB,NaN,Nicholas in Seattle,Tapes are exactly what I wanted. I'll be using...,Maxell Tapes,1410134400,tapes exactly wanted using seller,maxell tapes
1,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,...,True,2015-07-13,A2Z9HS4YPRDUWR,NaN,Mr. William H. Hamilton,LOVE THIS PRODUCT...NOT TOO READILY AVAILABLE ...,STILL WORKS GREAT!,1436745600,love product readily available anymore gratefu...,still works great
2,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,...,True,2015-06-24,AAGXCOQ8HSR95,NaN,ronald h sarnacki,very good qualiry,Five Stars,1435104000,good qualiry,five stars
3,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,...,True,2015-01-09,A28YYCYMAX19BX,NaN,chiefknisley--retired,no good,One Star,1420761600,good,one star
4,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videota...",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,...,True,2014-11-22,A2HLEPPTHZDW1M,NaN,john mcdonald,The old camcorder worked great! I just needed...,The old camcorder still does what I need.,1416614400,old camcorder worked great needed new blank tapes,old camcorder still need


### Lemmatization

In [19]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
# reviews['reviewClean'].apply(lambda x: [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(x)])

In [20]:
df_merge['reviewCleanLemm'] = df_merge['reviewClean'].apply(lambda x: [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(x)]).apply(lambda x:" ".join(x))

In [ ]:
df_merge.head()

# Sentiment analysis using pre-trained models

In [43]:
## Sentiment scores

# Textblob
df_merge['tb_score'] = df_merge['reviewCleanLemm'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Affin score
af = Afinn()
df_merge['afinn_score'] = df_merge['reviewCleanLemm'].apply(lambda x: af.score(x))

#vader analysis
sid = SentimentIntensityAnalyzer()
df_merge['vader_com'] = df_merge['reviewCleanLemm'].apply(lambda x: sid.polarity_scores(x)['compound'])
df_merge['vader_pos'] = df_merge['reviewCleanLemm'].apply(lambda x: sid.polarity_scores(x)['pos'])
df_merge['vader_neg'] = df_merge['reviewCleanLemm'].apply(lambda x: sid.polarity_scores(x)['neg']*(-1))
df_merge['vader_neu'] = df_merge['reviewCleanLemm'].apply(lambda x: sid.polarity_scores(x)['neu'])

In [45]:
df_merge.head(20)

,Unnamed: 0_x,index,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details,rankElectronics,Unnamed: 0_y,Rating,vote,verified,reviewTime,reviewerID,style,reviewerName,reviewText,summary,unixReviewTime,reviewClean,summaryClean,reviewCleanLemm,tb_score,afinn_score,vader_com,vader_pos,vader_neg,vader_neu
0,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videotape that offers outstanding performance in 8-mm camcorders. Pack of three (3) Maxell GX-MP 8mm Camcorder tapes (P6-120 GX), not three individuals tapes.']",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotape', 'Designed for use with video equipment that bears the 8 mark.', 'Storage Capacity: 120Minute(s) 8mm']","['>#15,985 in Electronics (See Top 100 in Electronics)', '>#17,595 in Electronics > Computers & Accessories']",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5654,5.0,NaN,True,2014-09-08,A2PUOBJDETBPQB,NaN,Nicholas in Seattle,Tapes are exactly what I wanted. I'll be using this seller again.,Maxell Tapes,1410134400,tapes exactly wanted using seller,maxell tapes,tape exactly want use seller,0.250000,1.0,0.0772,0.245,-0.000,0.755
1,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videotape that offers outstanding performance in 8-mm camcorders. Pack of three (3) Maxell GX-MP 8mm Camcorder tapes (P6-120 GX), not three individuals tapes.']",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotape', 'Designed for use with video equipment that bears the 8 mark.', 'Storage Capacity: 120Minute(s) 8mm']","['>#15,985 in Electronics (See Top 100 in Electronics)', '>#17,595 in Electronics > Computers & Accessories']",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5655,5.0,NaN,True,2015-07-13,A2Z9HS4YPRDUWR,NaN,Mr. William H. Hamilton,LOVE THIS PRODUCT...NOT TOO READILY AVAILABLE ANYMORE...SO I'M GRATEFUL TO BE ABLE TO STILL FIND IT.,STILL WORKS GREAT!,1436745600,love product not readily available anymore grateful able still find,still works great,love product not readily available anymore grateful able still find,0.266667,6.0,0.8020,0.474,-0.000,0.526
2,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videotape that offers outstanding performance in 8-mm camcorders. Pack of three (3) Maxell GX-MP 8mm Camcorder tapes (P6-120 GX), not three individuals tapes.']",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotape', 'Designed for use with video equipment that bears the 8 mark.', 'Storage Capacity: 120Minute(s) 8mm']","['>#15,985 in Electronics (See Top 100 in Electronics)', '>#17,595 in Electronics > Computers & Accessories']",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5656,5.0,NaN,True,2015-06-24,AAGXCOQ8HSR95,NaN,ronald h sarnacki,very good qualiry,Five Stars,1435104000,good qualiry,five stars,good qualiry,0.700000,3.0,0.4404,0.744,-0.000,0.256
3,11,2418,"['Electronics', 'Computers & Accessories']",NaN,"['High quality, extremely durable 8-mm videotape that offers outstanding performance in 8-mm camcorders. Pack of three (3) Maxell GX-MP 8mm Camcorder tapes (P6-120 GX), not three individuals tapes.']",NaN,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",[],NaN,Maxell,"['High quality, extremely durable 8mm videotape', 'Designed for use with video equipment that bears the 8 mark.', 'Storage Capacity: 120Minute(s) 8mm']","['>#15,985 in Electronics (See Top 100 in Electronics)', '>#17,595 in Electronics > Computers & Accessories']",[],Home Audio & Theater,NaN,"September 4, 1973",35.89,B00000J1EI,{},15985.0,5657,1.0,NaN,True,2015-01-09,A28YYCYMAX19BX,NaN,chiefknisley--retired,no good,One Star,1420761600,no good,one star,no go

# Sample 50k rows for labelling

In [57]:
# df_merge.to_csv('basic analysis.csv')
# sample 50k from the data

df_split = df_merge.dropna(subset=['summaryClean', 'reviewClean', 'reviewCleanLemm'])
first_split = df_split.sample(50000)
second_split = df_split.drop(first_split.index)

In [ ]:
first_split.to_csv('to_label_manually.csv')
second_split.to_csv('to_label_with_model.csv')